# 1. Acquisition


In [141]:
from sqlalchemy import create_engine
import pandas as pd
import requests

def acquire():

    # 1/3 Connection to the db

    sqlitedb_rel_path = '../data/processed/raw_data_project_m1.db'
    conn_str = f'sqlite:///{sqlitedb_rel_path}'
    engine = create_engine(conn_str)

    # Use dBeaver to cleanup a few fields i.e. gender, then make a DataFrame:

    sql_query = """ 
    SELECT country_info.uuid,
           country_info.country_code,
           career_info.dem_education_level,
           career_info.normalized_job_code, 
           personal_info.age, 
           personal_info.gender,
           poll_info.question_bbi_2016wave4_basicincome_vote,
           poll_info.question_bbi_2016wave4_basicincome_argumentsagainst,
       poll_info.question_bbi_2016wave4_basicincome_argumentsfor 
    FROM country_info
    JOIN career_info ON country_info.uuid = career_info.uuid
    JOIN personal_info ON personal_info.uuid = career_info.uuid
    JOIN poll_info ON poll_info.uuid = personal_info.uuid
    """

    poll_db = pd.read_sql_query(sql_query, engine)

    poll_job_code = poll_db['normalized_job_code'].unique().tolist()

    # 2/3 Extract job skills and description table using APIs
    # Make a list with job ids from the csv, and use it to get job names from website (open skills api)

    poll_db = pd.read_sql_query(sql_query, engine)

    poll_job_code = poll_db['normalized_job_code'].unique().tolist()

    url_list = []
    for i in poll_job_code:
        url_list.append(f'http://api.dataatwork.org/v1/jobs/{i}/related_skills')

    # Make a DataFrame with the collection of results.

    lst = []
    for url in url_list:
        response = requests.get(url)
        json_data = response.json()
        lst.append(json_data)

    api_skills = pd.DataFrame(lst)

    # Remove nulls

    api_skills.drop(index=api_skills[api_skills.skills.isnull()].index, inplace=True)

    #Formula to get the first skill, which is always the one with highest 'importance' value

    def get_max(x):
        top_skill = pd.json_normalize(x).iloc[0]['skill_name']
        return top_skill

    api_skills['top_skill'] = api_skills['skills'].apply(get_max)

    # 3/3 Get country names with web scraping

    url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
    html = requests.get(url).content

    # Make the soup

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Assign the correct [0] table data to variable tablle

    table = soup.find_all('table')[0]

    # Remove anything that isn't countries or acronyms

    items = [x.text for x in table.find_all('td')]

    # Create a dict of 2 lists (countries and their acronyms), and convert to DF

    countries = []
    acronyms = []
    for i in items:
        if i.startswith('('):
            acronyms.append(i[1:-2])
        else:
            countries.append(i[:-1])

    raw_dict = {'countries': countries, 'acronyms': acronyms}

    # This dict, however is not exactly the right shape for a latter pd.replace,

    raw_dict_df = pd.DataFrame(raw_dict)
    new_dict = raw_dict_df.set_index('acronyms').to_dict()
    countries_dict = new_dict['countries']

    # Adjust a couple keys missing in the countries table

    countries_dict['GB'] = 'Great Britain'
    countries_dict['GR'] = 'Greece'
    del countries_dict['EL']

    # From wrangling, include the lines to lead to a single table output
    # Merge poll file with api job list

    poll_api_merge = poll_db.merge(api_skills, left_on='normalized_job_code', right_on='job_uuid')

    # Map countries from countries_dict

    poll_api_merge_countries = poll_api_merge.fillna('Unemployed').replace({"country_code": countries_dict})
    
    poll_api_merge_countries['dem_education_level'] = poll_api_merge_countries['dem_education_level'].str.replace('Unemployed','no')

    return poll_api_merge_countries


In [142]:
aquired_result = acquire()
aquired_result
    

,uuid,country_code,dem_education_level,normalized_job_code,age,gender,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_argumentsagainst,question_bbi_2016wave4_basicincome_argumentsfor,error,job_uuid,job_title,normalized_job_title,skills,top_skill
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,Austria,high,861a9b9151e11362eb3c77ca914172d0,57 years old,male,I would probably vote for it,It might encourage people to stop working,It increases appreciation for household work a...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,Austria,high,861a9b9151e11362eb3c77ca914172d0,48 years old,female,I would vote for it,None of the above,It reduces anxiety about financing basic needs...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,Belgium,high,861a9b9151e11362eb3c77ca914172d0,43 years old,male,I would vote for it,It increases dependence on the state,It reduces anxiety about financing basic needs...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
3,a6361230-da58-0133-cd5a-0a81e8b09a82,Belgium,medium,861a9b9151e11362eb3c77ca914172d0,43 years old,male,I would probably vote against it,Only the people who need it most should get so...,None of the above,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,Bulgaria,high,861a9b9151e11362eb3c77ca914172d0,52 years old,female,I would probably vote for it,It might encourage people to stop working,It reduces anxiety about financing basic needs,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics


In [148]:
aquired_result

,uuid,country_code,dem_education_level,normalized_job_code,age,gender,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_argumentsagainst,question_bbi_2016wave4_basicincome_argumentsfor,error,job_uuid,job_title,normalized_job_title,skills,top_skill
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,Austria,high,861a9b9151e11362eb3c77ca914172d0,57 years old,male,I would probably vote for it,It might encourage people to stop working,It increases appreciation for household work a...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
1,cf24ed60-da3f-0133-0034-0a81e8b09a82,Austria,high,861a9b9151e11362eb3c77ca914172d0,48 years old,female,I would vote for it,None of the above,It reduces anxiety about financing basic needs...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
2,9ffc7390-dba5-0133-259b-0a81e8b09a82,Belgium,high,861a9b9151e11362eb3c77ca914172d0,43 years old,male,I would vote for it,It increases dependence on the state,It reduces anxiety about financing basic needs...,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
3,a6361230-da58-0133-cd5a-0a81e8b09a82,Belgium,medium,861a9b9151e11362eb3c77ca914172d0,43 years old,male,I would probably vote against it,Only the people who need it most should get so...,None of the above,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
4,1e2b4750-d987-0133-3c7c-0a81e8b09a82,Bulgaria,high,861a9b9151e11362eb3c77ca914172d0,52 years old,female,I would probably vote for it,It might encourage people to stop working,It reduces anxiety about financing basic needs,Unemployed,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...,computers and electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,Italy,high,4cee16550636e292b8d136486fce943b,58 years old,male,I would probably vote for it,Only the people who need it most should get so...,It reduces anxiety about financing basic needs...,Unemployed,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...,clerical
5106,c8c33390-da69-0133-063a-0a81e8b09a82,Poland,medium,4cee16550636e292b8d136486fce943b,44 years old,male,I would vote for it,It might encourage people to stop working | It...,It reduces anxiety about financing basic needs,Unemployed,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...,clerical
5107,d27d24d0-d9b1-0133-03d4-0a81e8b09a82,Poland,high,4cee16550636e292b8d136486fce943b,24 years old,male,I would vote for it,None of the above,It reduces anxiety about financing basic needs...,Unemployed,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...,clerical
5108,529f3080-d99a-0133-1b7b-0a81e8b09a82,Portugal,no,4cee16550636e292b8d136486fce943b,40 years old,male,I would probably vote for it,It is impossible to finance,It encourages financial independence and self-...,Unemployed,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...,clerical


## Bonus 2

In [140]:
from sqlalchemy import create_engine
import pandas as pd
import requests

sqlitedb_rel_path = '../data/processed/raw_data_project_m1.db'
conn_str = f'sqlite:///{sqlitedb_rel_path}'
engine = create_engine(conn_str)

# Use dBeaver to cleanup a few fields i.e. gender, then make a DataFrame:

sql_query = """ 
SELECT country_info.uuid,
        country_info.country_code,
        career_info.dem_education_level,
        career_info.normalized_job_code, 
        personal_info.age, 
        personal_info.gender,
        poll_info.question_bbi_2016wave4_basicincome_vote,
        poll_info.question_bbi_2016wave4_basicincome_argumentsagainst,
        poll_info.question_bbi_2016wave4_basicincome_argumentsfor 
FROM country_info
JOIN career_info ON country_info.uuid = career_info.uuid
JOIN personal_info ON personal_info.uuid = career_info.uuid
JOIN poll_info ON poll_info.uuid = personal_info.uuid
"""

poll_db = pd.read_sql_query(sql_query, engine)

poll_job_code = poll_db['normalized_job_code'].unique().tolist()

url_list = []
for i in poll_job_code:
    url_list.append(f'http://api.dataatwork.org/v1/jobs/{i}/related_skills')

# Make a DataFrame with the collection of results.

lst = []
for url in url_list:
    response = requests.get(url)
    json_data = response.json()
    lst.append(json_data)

api_skills = pd.DataFrame(lst)
    
# Remove nulls 

api_skills.drop(index=api_skills[api_skills.skills.isnull()].index, inplace=True)

#Formula to get the first skill, which is always the one with highest 'importance' value

def get_max(x):
    top_skill = pd.json_normalize(x).iloc[0]['skill_name']
    return top_skill

api_skills['top_skill'] = api_skills['skills'].apply(get_max)

api_skills


,error,job_uuid,job_title,normalized_job_title,skills
1,NaN,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...
2,NaN,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator,data coordinator,[{'skill_uuid': 'e1a8c649e57ad7b1cfeef1aad5ba5...
3,NaN,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer,database developer,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...
4,NaN,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist,data entry specialist,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...
5,NaN,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect,database architect,[{'skill_uuid': 'b3cb1294905e001d3d611bff1de39...
...,...,...,...,...,...
152,NaN,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner,analytical data miner,[{'skill_uuid': '20784bf09c9fe614603ad635e6093...
153,NaN,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,[{'skill_uuid': 'e72f8046dc4d704b6d1ca41dada93...
154,NaN,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk,data examination clerk,[{'skill_uuid': '500018df958f2b9f8b387cf5637a0...
155,NaN,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk,data entry clerk,[{'skill_uuid': '162cb662c7ef4dc9e86a2b5e88cbc...


In [136]:
bonus_2 = api_skills[['job_title', 'top_skill']]
bonus_2

,job_title,top_skill
1,Automatic Data Processing Planner,computers and electronics
2,Data Coordinator,information ordering
3,Database Developer,computers and electronics
4,Data Entry Specialist,clerical
5,Database Architect,computers and electronics
...,...,...
152,Analytical Data Miner,critical thinking
153,Data Analysis Assistant,mathematical reasoning
154,Data Examination Clerk,english language
155,Data Entry Clerk,clerical


## Bonus 2 wrangling

In [144]:
columns = ['dem_education_level', 'top_skill']
bonus_2 = aquired_result[columns]



bonus_2_grouped = bonus_2.groupby('dem_education_level').agg(list).reset_index()
bonus_2

,dem_education_level,top_skill
0,high,computers and electronics
1,high,computers and electronics
2,high,computers and electronics
3,medium,computers and electronics
4,high,computers and electronics
...,...,...
5105,high,clerical
5106,medium,clerical
5107,high,clerical
5108,no,clerical


## Bonus 2 analysis

In [145]:
from collections import Counter

def counter(x):
    return dict(Counter(x))

bonus_2_grouped['top_skill'] = bonus_2_grouped['top_skill'].apply(counter)

bonus_2_grouped

,dem_education_level,top_skill
0,high,"{'computers and electronics': 1659, 'informati..."
1,low,"{'computers and electronics': 513, 'informatio..."
2,medium,"{'computers and electronics': 1458, 'informati..."
3,no,"{'computers and electronics': 246, 'informatio..."


In [147]:
def get_tops(x):
    tops = sorted(x, key=x.get, reverse=True)[:10]
    return tops 
    
bonus_2_grouped['top_10_skills'] = bonus_2_grouped['top_skill'].apply(get_tops)

columns = ['dem_education_level', 'top_10_skills']
final = bonus_2_grouped[columns]

final

,dem_education_level,top_10_skills
0,high,"[computers and electronics, geography, clerica..."
1,low,"[computers and electronics, clerical, geograph..."
2,medium,"[computers and electronics, geography, informa..."
3,no,"[computers and electronics, geography, english..."
